In [2]:
# ==============================================
# CS-340: Project Two — Grazioso Salvare Dashboard
# Author: Lasupe Xiong
# ==============================================

# Jupyter-friendly Dash server
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

# Dash / viz / mapping
from dash import Dash, dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px

# Data + DB
import pandas as pd
import numpy as np
import base64
from crud import MongoConfig, CRUD  # my CRUD module from Project One

# ----------------------------------------------
# Configuration: MongoDB connection parameters
# ----------------------------------------------
AAC_USER   = "aacuser"
AAC_PASS   = "SNHU1234"   # replace if your password differs
AAC_HOST   = "localhost"
AAC_PORT   = 27017
AAC_AUTHDB = "aac"        # auth database
DB_NAME    = "aac"        # primary database
COLLECTION = "animals"    # collection with shelter outcomes

# Initialize CRUD with an authenticated attempt, then fallback to no-auth if needed.
def make_crud():
    try:
        cfg = MongoConfig(
            username=AAC_USER,
            password=AAC_PASS,
            host=AAC_HOST,
            port=AAC_PORT,
            authSource=AAC_AUTHDB
        )
        return CRUD(cfg, db_name=DB_NAME, collection_name=COLLECTION)
    except RuntimeError as e:
        print("[Auth Warning] Falling back to no-auth:", e)
        cfg = MongoConfig(host=AAC_HOST, port=AAC_PORT)
        return CRUD(cfg, db_name=DB_NAME, collection_name=COLLECTION)

db = make_crud()

# ----------------------------------------------
# Utility: load the Grazioso Salvare logo
# ----------------------------------------------
def load_logo_src(path="Grazioso Salvare Logo.png"):
    """Return a data URI for the logo so Dash can display it from local file."""
    try:
        with open(path, "rb") as f:
            encoded = base64.b64encode(f.read()).decode()
        return f"data:image/png;base64,{encoded}"
    except Exception:
        return None  # logo is optional; dashboard should still run

LOGO_SRC = load_logo_src()

# ----------------------------------------------
# Domain: rescue filter → MongoDB query builder
# ----------------------------------------------
# The client’s rescue profiles map to preferred breeds and age requirements.
# We keep the logic in one place so the controller (callbacks) stays clean.
TWO_YEARS_IN_WEEKS = 104

RESCUE_PROFILES = {
    "Water Rescue": {
        "breeds": [
            "Labrador Retriever", "Labrador Retriever Mix",
            "Chesapeake Bay Retriever", "Chesapeake Bay Retriever Mix",
            "Newfoundland", "Newfoundland Mix"
        ]
    },
    "Mountain or Wilderness Rescue": {
        "breeds": [
            "German Shepherd", "German Shepherd Mix",
            "Alaskan Malamute", "Alaskan Malamute Mix",
            "Siberian Husky", "Siberian Husky Mix"
        ]
    },
    "Disaster or Individual Tracking": {
        "breeds": [
            "Doberman Pinscher", "Doberman Pinscher Mix",
            "German Shepherd", "German Shepherd Mix",
            "Golden Retriever", "Golden Retriever Mix",
            "Bloodhound", "Bloodhound Mix"
        ]
    }
}

def build_query(profile_name: str | None):
    """
    Translate a rescue profile to a MongoDB query.
    For Reset / None, return an empty dict to fetch all documents.
    """
    if profile_name is None or profile_name == "Reset":
        return {}

    profile = RESCUE_PROFILES.get(profile_name)
    if not profile:
        return {}

    breeds = profile["breeds"]
    # Core SAR assumptions: dogs, under ~2 years old, matching preferred breeds.
    q = {
        "animal_type": "Dog",
        "age_upon_outcome_in_weeks": {"$lte": TWO_YEARS_IN_WEEKS},
        "breed": {"$in": breeds}
    }
    return q

# ----------------------------------------------
# Data access helpers
# ----------------------------------------------
def fetch_dataframe(query: dict) -> pd.DataFrame:
    """Run a Mongo query through CRUD.read and return a normalized DataFrame."""
    docs = db.read(query)
    df = pd.DataFrame(docs)
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    # Coerce known numeric columns if present
    for col in ["location_lat", "location_long", "age_upon_outcome_in_weeks"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

# Initial unfiltered data for first render
df_all = fetch_dataframe({})

# ----------------------------------------------
# Build the Dash app UI
# ----------------------------------------------
app = JupyterDash("GraziosoSalvare-ProjectTwo")

header_children = [
    html.Div([
        html.Div([
            html.Img(src=LOGO_SRC, style={"height": "90px"}) if LOGO_SRC else html.Div()
        ], style={"display": "inline-block", "verticalAlign": "middle", "marginRight": "16px"}),
        html.Div([
            html.H1("Grazioso Salvare Interactive Dashboard", style={"margin": 0}),
            html.P("Unique ID: LX-ProjectTwo-2025", style={"margin": 0, "fontStyle": "italic"})
        ], style={"display": "inline-block", "verticalAlign": "middle"})
    ], style={"textAlign": "center"})
]

filter_controls = html.Div([
    html.Hr(),
    html.H4("Rescue Filters"),
    dcc.RadioItems(
        id="rescue-filter",
        options=[
            {"label": "Water Rescue", "value": "Water Rescue"},
            {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
            {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
            {"label": "Reset (Show All)", "value": "Reset"},
        ],
        value="Reset",  # default to all data
        labelStyle={"display": "block", "marginBottom": "6px"}
    )
], style={"maxWidth": "420px", "margin": "0 auto"})

table_block = html.Div([
    html.Hr(),
    html.H4("Austin Animal Center Outcomes"),
    dash_table.DataTable(
        id="datatable",
        columns=[{"name": c, "id": c} for c in df_all.columns],
        data=df_all.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],  # default selection to support initial map render
        style_table={"overflowX": "auto", "maxHeight": "600px", "overflowY": "auto"},
        style_cell={"textAlign": "left", "minWidth": "120px", "width": "120px",
                    "maxWidth": "360px", "whiteSpace": "normal"}
    )
])

map_block = html.Div([
    html.Hr(),
    html.H4("Geolocation (selected animal)"),
    html.Div(id="map-container")
])

second_chart_block = html.Div([
    html.Hr(),
    html.H4("Top Breeds (current filter)"),
    dcc.Graph(id="breed-chart")
])

app.layout = html.Div(
    children=header_children + [filter_controls, table_block, map_block, second_chart_block],
    style={"padding": "16px"}
)

# ----------------------------------------------
# Callbacks (Controller)
# ----------------------------------------------

# 1) Update table from Mongo when the radio filter changes
@app.callback(
    Output("datatable", "data"),
    Output("datatable", "columns"),
    Input("rescue-filter", "value")
)
def refresh_table(profile_value):
    """
    Re-query MongoDB based on the selected rescue profile, then push rows/columns to DataTable.
    Keeping the DataTable's columns in sync prevents key errors if fields differ.
    """
    q = build_query(profile_value)
    dff = fetch_dataframe(q)
    if dff.empty:
        # provide a one-row placeholder so table renders cleanly
        dff = pd.DataFrame([{"status": "No records found for this filter."}])
    cols = [{"name": c, "id": c} for c in dff.columns]
    return dff.to_dict("records"), cols


# 2) Update the map whenever table data or selection changes
@app.callback(
    Output("map-container", "children"),
    Input("datatable", "derived_virtual_data"),
    Input("datatable", "derived_virtual_selected_rows")
)
def update_map(view_data, selected_rows):
    center = [30.75, -97.48]  # Austin
    if not view_data:
        return dl.Map(style={'width': '100%', 'height': '480px'}, center=center, zoom=10, children=[dl.TileLayer()])

    dff = pd.DataFrame(view_data)
    idx = 0 if not selected_rows else max(0, min(selected_rows[0], len(dff) - 1))

    lat = dff.iloc[idx].get("location_lat", np.nan)
    lon = dff.iloc[idx].get("location_long", np.nan)
    try:
        pos = [float(lat), float(lon)]
    except Exception:
        pos = center

    breed = str(dff.iloc[idx].get("breed", "Unknown breed"))
    name  = str(dff.iloc[idx].get("name", "Unknown name"))

    return dl.Map(
        style={'width': '100%', 'height': '480px'},
        center=center, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=pos, children=[
                dl.Tooltip(breed),
                dl.Popup([html.H5("Animal"), html.P(name)])
            ])
        ]
    )



# 3) Second chart (Top 10 breeds) — driven by the current table view
@app.callback(
    Output("breed-chart", "figure"),
    Input("datatable", "derived_virtual_data")
)
def update_breed_chart(view_data):
    import plotly.express as px
    if not view_data:
        return px.bar(pd.DataFrame({"breed": [], "count": []}), x="breed", y="count", title="No data")

    dff = pd.DataFrame(view_data)
    if "breed" not in dff.columns or dff.empty:
        return px.bar(pd.DataFrame({"breed": [], "count": []}), x="breed", y="count", title="No data")

    # Build a clean Top 10 table with explicit column names
    counts = (
        dff["breed"].astype(str).replace({"": "Unknown"}).fillna("Unknown")
        .value_counts().head(10)
    )
    top = pd.DataFrame({"breed": counts.index, "count": counts.values})

    fig = px.bar(top, x="breed", y="count", title="Top 10 Breeds (current selection)")
    fig.update_layout(xaxis_title="Breed", yaxis_title="Count", margin=dict(l=40, r=20, t=50, b=80))
    return fig


# Optional: style helper to highlight selected columns (nice UX touch)
@app.callback(
    Output("datatable", "style_data_conditional"),
    Input("datatable", "selected_columns")
)
def highlight_cols(selected_cols):
    return [{
        "if": {"column_id": c},
        "background_color": "#e8f6ff"
    } for c in (selected_cols or [])]


# ----------------------------------------------
# Run the app (Jupyter)
# ----------------------------------------------
app.run_server(debug=True, use_reloader=False, port=8050)


[Auth Warning] Falling back to no-auth: [MongoDB] Connection failed: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
Dash app running on https://tropicprovide-husbandocean-3000.codio.io/proxy/8050/
